In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow as tf

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate

# Define input layers
image_input1 = Input(shape=(140, 100, 3))
image_input2 = Input(shape=(140, 100, 3))
other_input = Input(shape=(6,))

# CNN branch for image processing
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input1)
pool1 = MaxPooling2D((2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu')(pool2)
pool3 = MaxPooling2D((2, 2))(conv3)
flatten1 = Flatten()(pool3)


conv4 = Conv2D(32, (3, 3), activation='relu')(image_input2)
pool4 = MaxPooling2D((2, 2))(conv4)
conv5 = Conv2D(64, (3, 3), activation='relu')(pool4)
pool5 = MaxPooling2D((2, 2))(conv5)
conv6 = Conv2D(128, (3, 3), activation='relu')(pool5)
pool6 = MaxPooling2D((2, 2))(conv6)
flatten2 = Flatten()(pool6)
# Concatenate the flattened outputs from the CNN branches
merged_cnn_outputs = Concatenate()([flatten1, flatten2])

# Concatenate the output of the MLP branch with the concatenated CNN outputs
merged_outputs = Concatenate()([merged_cnn_outputs, other_input])

# MLP branch for processing additional inputs
mlp_output = Dense(64, activation='relu')(merged_outputs)
mlp_output = Dense(128, activation='relu')(mlp_output)
mlp_output = Dense(128, activation='relu')(mlp_output)
mlp_output = Dense(64, activation='relu')(mlp_output)
mlp_output = Dense(32, activation='relu')(mlp_output)

# Final output layer
final_output = Dense(1, activation='sigmoid')(mlp_output)

# Create the model
model = tf.keras.Model(inputs=[image_input1, image_input2, other_input], outputs=final_output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
from PIL import Image
import numpy as np

csv_file_path = 'relations_full_links_shuffled.csv'
df = pd.read_csv(csv_file_path)

new_columns = ['color1', 'fabric1', 'category1', 'embedding1', 'color2', 'fabric2', 'category2', 'embedding2', 'result']
df_new = pd.DataFrame(columns=new_columns)

df_subset = df.iloc[21000:26000]
i = 0
for index, row in df_subset.iterrows():
    try:
        img_path = '../datathon/images/' + row['image1']
        img = Image.open(img_path)
        img = img.resize((100, 140))
        img_array1 = np.array(img) / 255.0
        #output1 = model.predict(img_array1)
    
        img_path = '../datathon/images/' + row['image2']
        img = Image.open(img_path)
        img = img.resize((100, 140))
        img_array2 = np.array(img) / 255.0
    except:
        continue
    new_row = {
        'color1': row['color1'], 
        'fabric1':  row['fabric1'],  
        'category1': row['category1'],
        'embedding1': img_array1,
        'color2': row['color2'], 
        'fabric2':  row['fabric2'],  
        'category2': row['category2'],
        'embedding2': img_array2,
        'result': row['result']
        #'image': embeeding(row['des_filename'])  # Assign a custom value 
    }
    df_new = pd.concat([df_new, pd.DataFrame([new_row])], ignore_index=True)
    
    i += 1
    if i %100 == 0:
        print(i)

print("New DataFrame with user-defined columns:")
print(df_new.head())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

numerical_features = df_new.iloc[:, [0, 1, 2, 4, 5, 6]]

images1 = np.array(df_new['embedding1'].tolist())
images2 = np.array(df_new['embedding2'].tolist())

numerical_features = np.asarray(numerical_features).astype(np.float32)

# Extract labels
labels = df_new.iloc[:, 8]
labels = np.asarray(labels).astype(np.float32)

print(images2.shape)

In [ ]:
from sklearn.model_selection import train_test_split

images1_train, images1_test, images2_train, images2_test, other_inputs_train, other_inputs_test, target_train, target_test = train_test_split(
    images1, images2, numerical_features, labels, test_size=0.05, random_state=42
)
print(images2_train.shape)


In [ ]:
##CONTINUE TRAINING
model = tf.keras.models.load_model('9')

In [ ]:
model.fit(
    [images1_train, images2_train, other_inputs_train],
    target_train,
    epochs=2,
)

In [ ]:
# Evaluate the model.
loss, accuracy = model.evaluate([images1_test, images2_test, other_inputs_test], target_test)

# Print the accuracy of the model.
print('Accuracy:', accuracy)

In [ ]:
model.save('10')